# **Natural Language Processing with Disaster Tweets ⛈**

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Proin semper, sem ut pulvinar pulvinar, est leo pretium sem, vel varius erat orci quis turpis. Aliquam augue orci, semper at mi vel, auctor fermentum ipsum. Nulla hendrerit hendrerit consequat. Vestibulum interdum, dolor id ultricies hendrerit, leo purus dignissim mauris, sit amet maximus nunc mauris eu mi. Quisque hendrerit ante eget pharetra iaculis. Nunc ullamcorper ante quam, non laoreet urna cursus eget. In hac habitasse platea dictumst. Donec ac risus consectetur, euismod magna sit amet, elementum justo. Sed non nisl lacinia, gravida odio ac, vehicula felis. Morbi non accumsan diam. Aenean nibh nunc, pulvinar quis vestibulum ac, tincidunt efficitur nibh. Maecenas vitae molestie leo. Pellentesque varius at felis vitae tristique. Nunc rutrum nisl sagittis velit finibus congue.

**Authors:**
- [Andrea Ramirez](https://github.com/Andrea-gt)
- [Adrian Flores](https://github.com/adrianRFlores)

***

## **(1) Import Libraries** ⬇️

In [2]:
#!pip install dataprep

In [3]:
# Data manipulation and visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import kurtosis, skew, probplot
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf
import itertools

# Standard libraries
import warnings
warnings.filterwarnings('ignore')

from dataprep.datasets import load_dataset
from dataprep.eda import create_report, plot, plot_correlation, plot_missing

# ===== ===== Reproducibility Seed ===== =====
# Set a fixed seed for the random number generator for reproducibility
random_state = 42

# Set matplotlib inline
%matplotlib inline

# Set default figure size
plt.rcParams['figure.figsize'] = (6, 4)

# Define custom color palette
palette = sns.color_palette("viridis", 12)

# Set the style of seaborn
sns.set(style="whitegrid")

## **(2) Data Upload** 📄

In [4]:
df = pd.read_csv('data/train.csv')
df.head(2)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


## **(3) Exploratory Analysis** 🔎

### **(1) Descripción General de los Datos**

In [5]:
# Print the number of records in the DataFrame
print("The given dataset has", df.shape[0], "registers and", df.shape[1], "columns.")

The given dataset has 7613 registers and 5 columns.


**Observaciones 💡 -->**

> - El conjunto de datos original cuenta con 7613 registros y 5 columnas, lo que indica que tiene una dimensión relativamente pequeña. Cada uno de los 7613 registros representa una observación única, mientras que las 5 columnas corresponden a diferentes características o variables medidas para cada observación, incluyendo el texto de un tweet, una palabra clave asociada y la ubicación desde donde se envió, aunque estas últimas dos pueden estar en blanco en algunas ocasiones.

**Fuente:** [Página oficial de Kaggle](https://www.kaggle.com/competitions/nlp-getting-started/data?select=train.csv)

In [6]:
# Basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Como se puede observar, se cuentan con 5 columnas o features en este conjunto de datos, siendo estas las que se describen a continuación.

- **id**: un identificador único para cada tweet.
- **text**: el texto del tweet.
- **location**: la ubicación desde donde se envió el tweet (puede estar en blanco).
- **keyword**: una palabra clave particular del tweet (puede estar en blanco).
- **target**: en el archivo `train.csv` solamente, indica si un tweet es sobre un desastre real (`1`) o no (`0`).

### **(2) Clasificación de las Variables**

| **Nombre**    | **Descripción**                                             | **Tipo de variable**                     |
|---------------|-------------------------------------------------------------|------------------------------------------|
| **id**        | Un identificador único para cada tweet                      | Cuantitativa                             |
| **text**      | El texto del tweet                                          | Cualitativa (descriptiva)                |
| **location**  | La ubicación desde donde se envió el tweet                  | Cualitativa (descriptiva)                |
| **keyword**   | Una palabra clave particular del tweet                      | Cualitativa (descriptiva)                |
| **target**    | Indica si un tweet es sobre un desastre real o no           | Cualitativa (binaria)                    |

**Observaciones 💡 -->**

> - El conjunto de datos posee 3 variables cualitativas descriptivas y 1 de tipo binaria.
> - La última variable del conjunto de datos (id) es de tipo cuantitativo.

### **(3) Exploración y Limpieza Inicial de los Datos**